# Analisi del prezzo delle azioni di Apple

Andiamo ad analizzare il titolo azionario di Apple (AAPL); nello specifico cercheremo risposta per le seguenti domande:

* **Qual è stato il valore massimo raggiunto dal AAPL ? In che data lo ha raggiunto ?**
* **Qual è stato il valore minimo raggiunto dal AAPL dopo il 2000 ? In che data lo ha raggiunto ?**
* **Qual è la percentuale di giorni in cui il prezzo di chiusura è stato inferiore ai 100 USD ?**
* **Qual è la percentuale di giorni dopo il 2014 in cui il prezzo di chiusura è stato inferiore ai 100 USD ?**
* **Visualizza il valore massimo per anno.**
* **In quale anno sono state scambiate più azioni di Apple (anno con il volume maggiore).**

### Importazione delle librerie

In [1]:
import os
import pandas as pd

In [2]:
import pyspark.sql.functions
from pyspark.sql.types import *

### Inizializzazione di Spark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()
from pyspark.sql.types import *

### Caricamento del dataset

In [4]:
apple = spark.read.csv('5 -  Serie storiche/data/AAPL.csv',header=True, inferSchema=True)

In [5]:
apple.show(5)

+----------+--------+--------+--------+--------+---------+---------+
|      Date|    Open|    High|     Low|   Close|Adj Close|   Volume|
+----------+--------+--------+--------+--------+---------+---------+
|1980-12-12|0.513393|0.515625|0.513393|0.513393| 0.410525|117258400|
|1980-12-15|0.488839|0.488839|0.486607|0.486607| 0.389106| 43971200|
|1980-12-16|0.453125|0.453125|0.450893|0.450893| 0.360548| 26432000|
|1980-12-17|0.462054|0.464286|0.462054|0.462054| 0.369472| 21610400|
|1980-12-18|0.475446|0.477679|0.475446|0.475446| 0.380182| 18362400|
+----------+--------+--------+--------+--------+---------+---------+
only showing top 5 rows



In [6]:
apple.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)



Cambiamo un po' di tipi.

In [7]:
data_schema = [StructField('Date',DateType(),nullable = True),
               StructField('Open',DoubleType(),nullable = True),
               StructField('High',DoubleType(),nullable = True),
               StructField('Low',DoubleType(),nullable = True),
               StructField('Close',DoubleType(),nullable = True),
               StructField('Adj Close',DoubleType(),nullable = True),
               StructField('Volume',IntegerType(),nullable = True)]

schema = StructType(fields = data_schema)

In [8]:
apple = spark.read.csv('5 -  Serie storiche/data/AAPL.csv',header=True,schema = schema)

In [9]:
apple.show(5)

+----------+--------+--------+--------+--------+---------+---------+
|      Date|    Open|    High|     Low|   Close|Adj Close|   Volume|
+----------+--------+--------+--------+--------+---------+---------+
|1980-12-12|0.513393|0.515625|0.513393|0.513393| 0.410525|117258400|
|1980-12-15|0.488839|0.488839|0.486607|0.486607| 0.389106| 43971200|
|1980-12-16|0.453125|0.453125|0.450893|0.450893| 0.360548| 26432000|
|1980-12-17|0.462054|0.464286|0.462054|0.462054| 0.369472| 21610400|
|1980-12-18|0.475446|0.477679|0.475446|0.475446| 0.380182| 18362400|
+----------+--------+--------+--------+--------+---------+---------+
only showing top 5 rows



In [10]:
apple.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



#### Qual è stato il valore massimo raggiunto dal AAPL ? In che data lo ha raggiunto ?

In [11]:
from pyspark.sql.functions import count, max, col, year, sum

In [12]:
apple.sort('High', ascending = False).show(10)

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2018-10-03|230.050003|233.470001|229.779999|232.070007| 229.39209|28654800|
|2018-10-04|230.779999|232.350006|226.729996|227.990005|225.359177|32042000|
|2018-10-02|    227.25|     230.0|226.630005|229.279999|226.634293|24788200|
|2018-09-05|228.990005|229.669998|225.100006|226.869995| 224.25209|33333000|
|2018-10-01|227.949997|229.419998|226.350006|227.259995|224.637604|23600800|
|2018-09-04|228.410004|229.179993|226.630005|228.360001|225.724899|27390100|
|2018-08-31|226.509995|228.869995|     226.0|227.630005|225.003342|43340100|
|2018-10-05|227.960007|228.410004|220.580002|224.289993|221.701859|33580500|
|2018-09-13|223.520004|228.350006|222.570007|226.410004|223.797409|41706400|
|2018-08-30|    223.25|228.259995|222.399994|225.029999|222.433319|48793800|

Come si vede sopra, il valore massimo è **233.47 $**, raggiunto il **3 ottobre 2018**.

In [13]:
apple.sort('Low', ascending = True).show(10)

+----------+--------+--------+--------+--------+---------+--------+
|      Date|    Open|    High|     Low|   Close|Adj Close|  Volume|
+----------+--------+--------+--------+--------+---------+--------+
|1981-08-10|    null|    null|    null|    null|     null|    null|
|1982-07-08|0.198661|0.198661|0.196429|0.196429|  0.15707|41081600|
|1982-07-09|0.203125|0.205357|0.203125|0.203125| 0.162425|32104800|
|1982-07-06|0.207589|0.207589|0.205357|0.205357|  0.16421|21924000|
|1982-07-07|0.205357|0.207589|0.205357|0.205357|  0.16421| 7593600|
|1982-07-12|0.207589|0.209821|0.207589|0.207589| 0.165995|15848000|
|1982-07-02|0.216518|0.216518|0.214286|0.214286|  0.17135|14526400|
|1982-08-06|0.220982|0.220982| 0.21875| 0.21875| 0.174919|24208800|
|1982-07-13|0.220982|0.223214|0.220982|0.220982| 0.176704|28593600|
|1982-08-05|0.223214|0.223214|0.220982|0.220982| 0.176704|17438400|
+----------+--------+--------+--------+--------+---------+--------+
only showing top 10 rows



Come si vede sopra, il valore minimo è **0.19 $**, raggiunto l'**8 luglio 1982**.

#### Qual è stato il valore minimo raggiunto dal AAPL dopo il 2000 ? In che data lo ha raggiunto ?

In [14]:
apple.filter(apple.Date > '2000-01-01').sort('Low', ascending = True).show(10)

+----------+--------+--------+--------+--------+---------+---------+
|      Date|    Open|    High|     Low|   Close|Adj Close|   Volume|
+----------+--------+--------+--------+--------+---------+---------+
|2003-04-17|0.942857|0.946429|0.908571|0.937143| 0.820964|154064400|
|2003-04-16|0.927857|0.976429|0.922857|0.945714| 0.828473|254044000|
|2003-04-11|1.003571|1.031429|0.923571|0.942857|  0.82597|348177200|
|2003-04-21|0.937857|0.942143|0.927143|0.938571| 0.822216| 38080000|
|2003-04-24|0.965714|0.972143|0.928571|    0.96| 0.840988| 81277000|
|2003-04-22|0.941429|0.972857|   0.935|   0.965| 0.845368| 75142200|
|2003-04-25|0.961429|    0.97|   0.945|0.953571| 0.835356| 51329600|
|2003-04-15|0.970714|0.971429|    0.95|0.956429| 0.837859| 75992000|
|2002-10-08|0.992857|0.997143|0.954286|0.977143| 0.856006|113411200|
|2003-04-23|0.966429|0.973571|0.954286|    0.97| 0.849748| 52420200|
+----------+--------+--------+--------+--------+---------+---------+
only showing top 10 rows



Il valore minimo di Apple dopo il 2000 è stato raggiunto il **17 aprile 2003**, ed è **0.90 $**.

#### Qual è la percentuale di giorni in cui il prezzo di chiusura è stato inferiore ai 100 USD ?

Numero di giorni in cui il prezzo di chiusura è stato inferiore a 100.

In [15]:
apple.filter(apple.Close < 100).count()

8604

Numero totale di giorni.

In [16]:
apple.count()

9595

In [17]:
print('La percentuale di giorni in cui il prezzo di chiusura è stato inferiore a 100$ è ', str(round((8604/9595)*100,2)),'%')

La percentuale di giorni in cui il prezzo di chiusura è stato inferiore a 100$ è  89.67 %


#### Qual è la percentuale di giorni dopo il 2014 in cui il prezzo di chiusura è stato inferiore ai 100 USD ?

Numero di giorni dopo il 2014 dove il prezzo di chiusura è stato inferiore a 100$.

In [18]:
apple.filter("Date > '2013-12-31' and Close < 100").count()

271

Numero di giorni dopo il 2014.

In [19]:
apple.filter("Date > '2013-12-31'").count()

1258

In [20]:
print('La percentuale di giorni dopo il 2014 in cui il prezzo di chiusura è stato inferiore ai 100 USD è ',str(round((271/1258)*100,2)),'%')

La percentuale di giorni dopo il 2014 in cui il prezzo di chiusura è stato inferiore ai 100 USD è  21.54 %


#### Visualizza il valore massimo per anno.

Estraggo l'anno dalla colonna 'Date' e lo inserisco in nuova colonna **'year'**.

In [21]:
apple = apple.withColumn('year',year(apple['Date']))

In [22]:
apple.show(5)

+----------+--------+--------+--------+--------+---------+---------+----+
|      Date|    Open|    High|     Low|   Close|Adj Close|   Volume|year|
+----------+--------+--------+--------+--------+---------+---------+----+
|1980-12-12|0.513393|0.515625|0.513393|0.513393| 0.410525|117258400|1980|
|1980-12-15|0.488839|0.488839|0.486607|0.486607| 0.389106| 43971200|1980|
|1980-12-16|0.453125|0.453125|0.450893|0.450893| 0.360548| 26432000|1980|
|1980-12-17|0.462054|0.464286|0.462054|0.462054| 0.369472| 21610400|1980|
|1980-12-18|0.475446|0.477679|0.475446|0.475446| 0.380182| 18362400|1980|
+----------+--------+--------+--------+--------+---------+---------+----+
only showing top 5 rows



In [23]:
apple.groupBy('year').agg(max('High').alias('Max_Value')).sort('year',ascending = False).show(30)

+----+----------+
|year| Max_Value|
+----+----------+
|2018|233.470001|
|2017|177.199997|
|2016|118.690002|
|2015|134.539993|
|2014|    119.75|
|2013| 82.162857|
|2012|100.724289|
|2011| 60.957142|
|2010| 46.665714|
|2009| 30.564285|
|2008| 28.608572|
|2007| 28.994286|
|2006| 13.308572|
|2005|     10.78|
|2004|  4.969285|
|2003|  1.786429|
|2002|  1.869286|
|2001|  1.937143|
|2000|  5.370536|
|1999|  4.214286|
|1998|    1.5625|
|1997|  1.055804|
|1996|  1.267857|
|1995|  1.790179|
|1994|    1.5625|
|1993|  2.330357|
|1992|       2.5|
|1991|  2.616071|
|1990|  1.705357|
|1989|  1.799107|
+----+----------+
only showing top 30 rows



#### In quale anno sono state scambiate più azioni di Apple (anno con il volume maggiore).

In [24]:
apple_years = apple.withColumn('year',year(apple['Date'])).drop('Date')

In [25]:
apple_years.show(5)

+--------+--------+--------+--------+---------+---------+----+
|    Open|    High|     Low|   Close|Adj Close|   Volume|year|
+--------+--------+--------+--------+---------+---------+----+
|0.513393|0.515625|0.513393|0.513393| 0.410525|117258400|1980|
|0.488839|0.488839|0.486607|0.486607| 0.389106| 43971200|1980|
|0.453125|0.453125|0.450893|0.450893| 0.360548| 26432000|1980|
|0.462054|0.464286|0.462054|0.462054| 0.369472| 21610400|1980|
|0.475446|0.477679|0.475446|0.475446| 0.380182| 18362400|1980|
+--------+--------+--------+--------+---------+---------+----+
only showing top 5 rows



In [26]:
apple_years.groupBy('year') \
.agg(sum('Volume').alias('Total_Year_Volume')) \
.sort('Total_Year_Volume',ascending = False) \
.show(40)

+----+-----------------+
|year|Total_Year_Volume|
+----+-----------------+
|2008|      71495301500|
|2007|      61748996400|
|2006|      53924741500|
|2005|      45600245600|
|2010|      37756231800|
|2009|      35813421700|
|1999|      34275676400|
|2012|      32991051100|
|2011|      31014834900|
|2004|      30450417200|
|2000|      30075399200|
|1998|      28798548800|
|2013|      25605392400|
|2001|      23664449200|
|2002|      19253481800|
|1995|      18566634800|
|1997|      17990840000|
|2003|      17807563200|
|2014|      15914488100|
|1987|      14942827200|
|1991|      14336912800|
|1994|      14288974000|
|1993|      14113232000|
|1986|      13330805600|
|1996|      13298555200|
|2015|      13066049900|
|1989|      12726456400|
|1985|      11373068000|
|1983|      11128252800|
|1990|      11100485200|
|1984|      10494758400|
|1988|      10323244400|
|1992|      10284478400|
|2016|       9682477800|
|2018|       8539036200|
|2017|       6810776500|
|1982|       5341252000|


L'anno con il volume maggiore è stato il 2008.